In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from os import listdir, mkdir, remove, system
from os.path import isfile, join
from datetime import datetime, date, timedelta

In [3]:
# Import files (should be downloaded ito the same directory as this script)

file_pattern = "LvpsThresMonitor_ATLTILLV0"
path="./"
files = [f for f in listdir(path) if isfile(join(path, f)) and file_pattern in f]
print(files)

['LvpsThresMonitor_ATLTILLV00.txt', 'LvpsThresMonitor_ATLTILLV01.txt', 'LvpsThresMonitor_ATLTILLV02.txt', 'LvpsThresMonitor_ATLTILLV03.txt']


In [4]:
# Date and time
startdate_ = "2022.07.04 00:00:00.00"
enddate_ = "2022.11.28 12:00:00.00"


startdate = datetime.strptime(startdate_, "%Y.%m.%d %H:%M:%S.%f")
enddate = datetime.strptime(enddate_, "%Y.%m.%d %H:%M:%S.%f")
print(startdate)
print(enddate)
print(startdate<enddate)

2022-07-04 00:00:00
2022-11-28 12:00:00
True


In [5]:
# Read the files to determine LVPS of interest
# Get LVPS list to input to getDataSafely method
LVPS_data = []
for f in files:
    print(f)
    # Hard code values: EBA 00, LBA 01, LBC 02, EBC 03:
    if "00" in f: module="EBA"
    elif "01" in f: module="LBA"
    elif "02" in f: module="LBC"
    elif "03" in f: module="EBC"

    # List of LVPS
    LVPS_list = []

    
    file = open(f,"r")

    for line in file:
        # Do not read the lines with just single values
        # We want the lines that contain dates and thresholds
        if len(line.split(", ")) > 1:
            time = datetime.strptime(line.split(", ")[0], "%Y.%m.%d %H:%M:%S.%f")
            LVPS = str(line.split(", ")[1][5:])
            I_reading = float(line.split(", ")[2].split("= ")[1])
            I_threshold = float(line.split(", ")[3].split("= ")[1])
        
            if time >= startdate and time <= enddate:
                LVPS_list.append(LVPS)

    # Keep onlyunique values 
    LVPS_list = list(set(LVPS_list))
    for LVPS in LVPS_list:
        LVPS_data.append([module,LVPS])


LvpsThresMonitor_ATLTILLV00.txt
LvpsThresMonitor_ATLTILLV01.txt
LvpsThresMonitor_ATLTILLV02.txt
LvpsThresMonitor_ATLTILLV03.txt


In [6]:
# Retrieve data from online
# Setup folder to feed in data
datapath = "data"
print(listdir(path))
if datapath not in listdir(path):
    mkdir(datapath)
datapath = "./"+datapath+"/"

['data', 'LVPS-Burnout-Plot.ipynb', 'LVPS-Burnout-Plot.py', 'LvpsThresMonitor_ATLTILLV00.txt', 'LvpsThresMonitor_ATLTILLV01.txt', 'LvpsThresMonitor_ATLTILLV02.txt', 'LvpsThresMonitor_ATLTILLV03.txt', 'plots']


In [7]:
# # URL to retrieve data (do not touch)
# url = 'http://atlas-ddv.cern.ch:8089/multidata/getDataSafely'

# # Special function to split up time period to retrieve data
# def perdelta(start, end, delta):
#     curr = start
#     while curr < end:
#         yield curr, min(curr + delta, end)
#         curr += delta


# for module,LVPS in LVPS_data:
#     filenames=[]
#     counter=0
#     start = datetime.strptime(startdate_.split(" ")[0], "%Y.%m.%d")
#     end = datetime.strptime(enddate_.split(" ")[0], "%Y.%m.%d")

#     for s, e in perdelta(date(2022, 7, 4), date(2022, 11, 28), timedelta(days=30)):
#         counter+=1

#         #Format retrieval parameters (temporary filename and LVPS for single digit)
#         output_file=datapath+str(module)+str(LVPS)+"_"+str(counter)+".txt"
#         if int(LVPS)<10: LVPS = str(LVPS)[-1]

#         cmd = 'wget -O '+output_file+' --post-data "queryInfo=atlas_pvssTIL, comment_, TIL '+module+' LVPS '+LVPS+' MotherBoard 5V OUTPUT_I, ' + s.strftime("%d-%m-%Y") + ' 00:00, ' + e.strftime("%d-%m-%Y") + ' 00:00, , , , , ,no, , +1!" ' + url 
#         system(cmd) 
#         filenames.append(output_file)
#     print(filenames)
#     if int(LVPS)<10: LVPS = "0"+str(int(LVPS))
#     else: LVPS=str(LVPS)

#     new_output_file = datapath+module+LVPS+"_5VMotherboardCurrent.txt"
#     with open(new_output_file, 'w') as outfile:
#         for fname in filenames:
#             with open(fname) as infile:
#                 for line in infile:
#                     outfile.write(line)
#             remove(fname)

In [ ]:
# Plotting

# Retrieve data from online
# Setup folder to feed in data
plotpath = "plots"
print(listdir(datapath))
if plotpath not in listdir(path):
    mkdir(plotpath)
plotpath = "./"+plotpath+"/"

files = [f for f in listdir(datapath) if isfile(join(plotpath, f))]
print(files)

exit()
infile = open("LBA01.txt","r")
module = ["LBA","EBA","LBC","EBC"]
N=64
print(infile)
for mod in module:
    for j in range(64):
        i=j+1
        if i<10:istr="0"+str(i)
        else: istr=str(i)
        infile_name = mod+istr+".txt"
        infile=open(infile_name,"r")

        for line in infile:
            data = line.split(", ")[1:]
        # print(data)
        I,t =[],[]
        for i, entry in enumerate(data):
            if "LVPS" not in entry:
                current = entry.split(" ")[0]
                date_time = entry.split(" ")[1]+" "+entry.split(" ")[2]
                time = datetime.datetime.strptime(date_time, "%d-%m-%Y %H:%M:%S:%f")
                if i==0:
                    referencetime=datetime.datetime.strptime(date_time, "%d-%m-%Y %H:%M:%S:%f")
                delta = time-referencetime
                t.append(delta.total_seconds())
                I.append(np.around(float(current),decimals=2))
        plt.step(t,I, where="post")
        plt.ylabel("Current [A]")
        plt.xlabel("Time Since Beginning of Run 3 [s]")
        plt.suptitle(mod+istr)
        plt.tight_layout()
        plt.savefig(mod+istr+".png")
